In [2]:
import ipywidgets as w

In [3]:
intro_box = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="E.g. Workshop without masks", description="Simulation name:",
                                style={'description_width': '35%'}),
                         w.Text(placeholder="E.g. 17/R-033", description="Room number:",
                                style={'description_width': '35%'})),
                   layout=w.Layout(padding='0px 0px 50px 0px'))

In [4]:
volume_text = w.Text(placeholder="Room volume (m³)", description="Room volume")
area_text = w.Text(placeholder="Room floor area (m²)", description="Floor area")
height_text = w.Text(placeholder="Room ceiling height (m²)", description="Ceiling height")

room_dimensions_box = w.VBox(children=(volume_text, w.Label(value="------- OR -------"),area_text, height_text),
                             layout=w.Layout(padding='0px 0px 50px 0px'))

def manage_room_text_fields(_):
    volume_text.disabled = bool(area_text.value) or bool(height_text.value)
    area_text.disabled = bool(volume_text.value)
    height_text.disabled = bool(volume_text.value)

for text in [volume_text, area_text, height_text]:
    text.observe(manage_room_text_fields)
    text.style.description_width = "auto"

volume_text.observe(manage_room_text_fields)
area_text.observe(manage_room_text_fields)
height_text.observe(manage_room_text_fields)

In [5]:
ventilation_label = w.Label("Ventilation type:")
mechanical_button = w.ToggleButton(value=False, description='Mechanical', layout={'width': 'auto'})
natural_button = w.ToggleButton(value=False, description='Natural', layout={'width': 'auto'})

acph_text = w.Text(description="Air changes per hour", style={'description_width': 'auto'})
asfr_text = w.Text(description="Air supply flow rate", style={'description_width': 'auto'})

w.jsdlink((acph_text, 'value'), (asfr_text, 'disabled'))
w.jsdlink((asfr_text, 'value'), (acph_text, 'disabled'))

mech_vent_box = w.VBox(
    children=(acph_text, w.Label(value="------- OR -------"), asfr_text),
    layout=w.Layout(display='none')
)

nat_vent_box = w.VBox(
    children=(
        w.Text(description='Number of windows:', style={'description_width': 'auto'}),
        w.Text(description='Height of window:', placeholder='meters', style={'description_width': 'auto'}),
        w.Text(description='Width of window:', placeholder='meters', style={'description_width': 'auto'}),
        w.Text(description='Opening distance:', placeholder='centimeters', style={'description_width': 'auto'}),
        w.HBox(children=(
            w.Label('Windows open'),
            w.RadioButtons(options=('Always', '15 min / 2h'))
        ))
    ),
    layout=w.Layout(display='none'),
)



tab_bar = w.HBox(children=(mechanical_button, natural_button))
ventilation_type_menu = w.VBox(children=(ventilation_label, tab_bar), layout=w.Layout(padding='0px 0px 20px 0px'))
ventilation_box = w.VBox(children=(ventilation_type_menu, mech_vent_box, nat_vent_box),
                         layout=w.Layout(padding='0px 0px 50px 0px'))

def handle_natural_toggle(_):
    if natural_button.value:
        mechanical_button.value = False
        ventilation_box.children[2].layout.display = 'block'
    else:
        ventilation_box.children[2].layout.display = 'none'

def handle_mechanical_toggle(_):
    if mechanical_button.value:
        natural_button.value = False
        ventilation_box.children[1].layout.display = 'block'
    else:
        ventilation_box.children[1].layout.display = 'none'

natural_button.observe(handle_natural_toggle)
mechanical_button.observe(handle_mechanical_toggle)

In [7]:
event_data_label = w.Label("Event data:")
attendees_label = w.Label("Attendees:")
total_people_text = w.Text(description="Total number of people:", style={'description_width': 'auto'})
infected_people_text = w.Text(description="Number of infected people:", style={'description_width': 'auto'})
activity_dropdown = w.Dropdown(description="Activity type:", options=["Training", "Workshop", "Office"], value=None)
start_hour = w.Dropdown(options=["HH"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(24)], value="HH")
start_minute = w.Dropdown(options=["MM"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(60)], value="MM")
finish_hour = w.Dropdown(options=["HH"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(24)], value="HH")
finish_minute = w.Dropdown(options=["MM"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(60)], value="MM")
start_box = w.VBox(children=(w.Label("Start:"),
                             w.HBox(children=(start_hour, start_minute))),
                   layout=w.Layout(padding="0px 20px 0px 0px"))
finish_box = w.VBox(children=(w.Label("Finish:"),
                              w.HBox(children=(finish_hour, finish_minute))))
event_time_box = w.HBox(children=(start_box, finish_box))

event_intro_box = w.VBox(children=(
    event_data_label,
    attendees_label,
    total_people_text,
    infected_people_text,
    activity_dropdown,
    event_time_box
))

months = ("January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December")

single_button = w.ToggleButton(value=False, description='Single event', layout={'width': 'auto'})
recurrent_button = w.ToggleButton(value=False, description='Recurrent usage', layout={'width': 'auto'})
single_recurrent_selection_bar = w.HBox(children=(single_button, recurrent_button))
month_dropdown = w.Dropdown(options=months, value=None, description="Month:", layout=w.Layout(padding="0px 20px 0px 0px"))
day_dropdown = w.Dropdown(options=[i for i in range(1, 32)], value=None, description="Day:")
date_selector = w.HBox(children=(month_dropdown, day_dropdown), layout=w.Layout(display='none'))
months_bar = w.HBox(children=[w.ToggleButton(value=False, description=month[0]) for month in months],
                    layout = w.Layout(display='none'))

def handle_single_click(_):
    if single_button.value:
        date_selector.layout.display = 'contents'
        recurrent_button.value = False
    else:
        date_selector.layout.display = 'none'

def handle_recurrent_click(_):
    if recurrent_button.value:
        months_bar.layout.display = 'contents'
        single_button.value = False
    else:
        months_bar.layout.display = 'none'

single_button.observe(handle_single_click)
recurrent_button.observe(handle_recurrent_click)

date_box = w.VBox(children=(w.HBox(children=(single_button, recurrent_button)),
                            w.HBox(children=(date_selector, months_bar))))

lunch_start_hour = w.Dropdown(options=["HH"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(24)],
                              value="HH", disabled=True)
lunch_start_minute = w.Dropdown(options=["MM"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(60)],
                                value="MM", disabled=True)
lunch_finish_hour = w.Dropdown(options=["HH"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(24)],
                               value="HH", disabled=True)
lunch_finish_minute = w.Dropdown(options=["MM"] + [("0" + str(i)) if len(str(i)) == 1 else str(i) for i in range(60)],
                                 value="MM", disabled=True)
lunch_start_box = w.VBox(children=(w.Label("Start:"),
                                   w.HBox(children=(lunch_start_hour, lunch_start_minute))),
                         layout=w.Layout(padding="0px 20px 0px 0px"))
lunch_finish_box = w.VBox(children=(w.Label("Finish:"),
                                    w.HBox(children=(lunch_finish_minute, lunch_finish_minute))))
lunch_time_box = w.HBox(children=(lunch_start_box, lunch_finish_box))

lunch_button = w.ToggleButton(value=False, description="Lunch break")

def handle_lunch(_):
    for dropdown in (lunch_start_hour, lunch_start_minute, lunch_finish_hour, lunch_finish_minute):
        dropdown.disabled = not lunch_button.value

lunch_button.observe(handle_lunch)

break_box = w.VBox(children=(lunch_button, lunch_time_box))

